In [2]:
import pandas as pd
pd.options.display.max_rows=6
import numpy as np
from haversine import haversine
import json
stores = pd.read_csv('/Users/JayLiang/Desktop/Media Storm/Big Lots/Data/MediaStorm Data Extract - Store list.txt', sep='|')

In [3]:
stores_lat_lng = dict()
for index in stores.index:
    stores_lat_lng[stores.loc[index, 'location_id']] = (stores.loc[index, 'latitude_meas'], 
                                                        stores.loc[index, 'longitude_meas'])

In [9]:
location_id = list(stores_lat_lng.keys())
n = len(location_id)

distance = dict()
for i in range(n):
    p1 = stores_lat_lng[location_id[i]]
    other_stores = dict()
    for j in range(i+1, n):
        p2 = stores_lat_lng[location_id[j]]
        d = haversine(p1, p2, miles=True)
        other_stores[location_id[j]] = d
    distance[location_id[i]] = other_stores

In [22]:
distance_matrix = pd.DataFrame(distance).T

In [23]:
# fill nan fields
for column in distance_matrix.columns:
    for index in distance_matrix.index:
        if np.isnan(distance_matrix.loc[index, column]):
            if index == column:
                distance_matrix.set_value(index, column, 0)
            else:
                distance_matrix.set_value(index, column, distance_matrix.loc[column, index])

In [24]:
distance_matrix.head()

,3,29,30,32,37,42,43,45,49,51,...,5332,5333,5334,5335,5336,5338,5339,5343,5345,6990
1,451.289895,60.072235,107.328621,66.558476,86.125901,81.118759,123.103602,122.637525,88.782638,258.957367,...,281.845873,397.066600,435.732245,411.317694,927.138735,99.020239,248.736426,135.856051,248.855889,5843.308757
3,0.000000,408.738243,555.941985,449.854565,415.163011,371.261612,572.701649,573.230273,417.206036,451.045201,...,190.582797,537.234887,620.847863,393.956898,477.867001,550.283224,365.704356,561.754066,202.807091,5758.597666
29,408.738243,0.000000,162.861051,113.215604,34.422310,63.346293,177.151256,175.153527,118.836617,209.881656,...,253.405335,350.421403,397.016108,352.791044,880.732305,148.407891,190.374225,155.565830,206.513672,5795.161183
30,555.941985,162.861051,0.000000,117.314492,178.332377,184.809566,17.291264,21.243002,154.555948,327.292973,...,379.753010,454.211196,475.423232,498.365344,1032.899333,29.396749,338.184008,114.323729,354.501576,5889.447162
32,449.854565,113.215604,117.314492,0.000000,145.616092,91.662207,134.455039,138.053209,37.243409,321.597159,...,267.068330,461.112664,501.918627,463.435223,927.689710,125.789414,301.922144,189.330045,253.813292,5907.098606


In [25]:
def clustering(radius, distance_matrix):
    clusters = list()
    checked_stores = set()

    for index in distance_matrix.index:
        if index not in checked_stores:
            checked_stores.add(index)
            other_stores = distance_matrix.loc[index]
            cluster = set(other_stores[(other_stores < radius) & (other_stores > 0)].index)
            q = list(cluster)
            cluster.add(index)
            while len(q) > 0:
                curr_store = q.pop(0)
                if curr_store not in checked_stores:
                    checked_stores.add(curr_store)
                    other_stores = distance_matrix.loc[curr_store]
                    s = set(other_stores[(other_stores < radius) & (other_stores > 0)].index)
                    new_stores = s.difference(cluster)
                    if len(new_stores) > 0:
                        q += list(new_stores)
                        cluster = cluster.union()

            clusters.append(cluster)
    return clusters

In [27]:
range(5)

range(0, 5)

In [31]:
print(*zip(range(5)))

(0,) (1,) (2,) (3,) (4,)
